In [1]:
%matplotlib inline
import numpy as np
import pylab as pl
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import norm
import scipy.interpolate
import random

from numpy import random
import matplotlib.image as mpimg
from scipy.optimize import curve_fit

## Formula that describes the ellipse on the Armenteros-odolanski plot.
def func(x, m_Ks, m_pi):
    return np.sqrt(abs((-m_Ks)**2*(1.-x**2)/4.-m_pi**2))

In [2]:
toydata=np.loadtxt("ToyKS-0.txt",delimiter=',') #loading the actual collision data

In [3]:
nps=1 #Number of pseudo datasets.

ngi=100 #Number of iterations for calculating the Theil index for the same mass pair.

kaon=(550-450)*np.random.rand(nps)+450
pion=(200-100)*np.random.rand(nps)+100

y_binnum=500 #number of bins along the y-axis
x_binnum=y_binnum #number of bins along the x-axis

#Binning the toy data.
StartMatrix,ax,ay=np.histogram2d(toydata[:,1],toydata[:,0],x_binnum,range=[[0,350],[-1,1]])

Theil=np.zeros((nps,3)) #[Theil index,Kaon mass,pion mass]

# Creating the histogram of the toy data.
a = toydata[:,0]
counts, bins = np.histogram(a, bins=100, density=True)
cum_counts = np.cumsum(counts)
bin_widths = (bins[1:] - bins[:-1])

# Interpolating the histogram and finding the distribution.
x = cum_counts*bin_widths
y = bins[1:]
inverse_density_function = scipy.interpolate.interp1d(x, y)
    
for p in range(nps): #going trough the pseudo datasets
    
    #Generating alpha values using the same distribution.
    alpha=inverse_density_function( random.uniform( x[0], x[-1],[ngi,toydata.shape[0]]) )
    pt=func(alpha,kaon[p],pion[p]) #finding the y-coordinate of the bin where an event belongs
    TheilGaus=np.zeros((ngi))
    for jj in range(ngi):
        matrix,ax,ay=np.histogram2d(pt[jj],alpha[jj],x_binnum,range=[[0,350],[-1,1]])
        Matrix=matrix + StartMatrix+1

        T=0
        events=np.asarray(Matrix).reshape(-1)
        mean=np.mean(events)

        #calculating the Theil index

        events1=events
        Ev=events*np.log(events1/mean)
        T=np.sum(Ev)/mean
        T/=Ev.shape[0]*np.log(Ev.shape[0])

        TheilGaus[jj]=T

    mu, std = norm.fit(TheilGaus)
    Theil[p]=[mu,kaon[p],pion[p]] 
    
np.savetxt('Theilindex-final-bin500x500.txt',Theil, delimiter=',')    